In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import average_precision_score as avp
from statsmodels.stats.inter_rater import fleiss_kappa

In [2]:
DIR = "H:/Il mio Drive/Dottorato/Ricerca/Valutazioni/files/top_200/"
DIR1 = "H:/Il mio Drive/Dottorato/Ricerca/Valutazioni/Annotatori/"

In [3]:
def match_annotations(extr_file, annot_file):
    files = list()
    for f in (extr_file, annot_file):
        if ".csv" in f:
            files.append(pd.read_csv(f, delimiter=';'))
        elif ".xlsx" in f:
            files.append(pd.read_excel(f))
        else:
            raise Exception("File not supported")

    return pd.merge(*files)

def accuracy(dataframe, col="Valutazione", true_labels=["SI"]):
    arr = np.isin(np.array(dataframe[col]), true_labels)
    return np.sum(arr)/len(arr)

def average_p(dataframe, col="Valutazione", true_labels=["SI"]):
    y_score = dataframe['score']
    y_true = np.isin(np.array(dataframe[col]), true_labels)
    return avp(y_true, y_score)

def fleiss(files, col="Valutazione"):
    data = list()
    cat = set()
    for f in files:
        df = pd.read_excel(f)
        df.fillna('')
        data.append(df[col].tolist())
        cat = cat.union(set(df[col].unique()))
    table = list()
    for d in zip(*data):
        table.append([d.count(c) for c in cat])
    return fleiss_kappa(table)


In [29]:
filein = 'sketch_mwe.csv'
file = DIR + filein
file1 = DIR1 + 'tutti.xlsx'
df = match_annotations(file, file1)
print(filein)

print("\tGeneral")
print("acc: ", accuracy(df))
print("avp: ", average_p(df))

print("\tWaste")
print("acc: ", accuracy(df, "Dominio", ["WASTE", "WASTE_LAW", "WASTE_REL"]))
print("avp: ", average_p(df, "Dominio", ["WASTE", "WASTE_LAW", "WASTE_REL"]))

print("\tLaw")
print("acc: ", accuracy(df, "Dominio", ["LAW", "OTHER"]))
print("avp: ", average_p(df, "Dominio", ["LAW", "OTHER"]))

sketch_mwe.csv
	General
acc:  0.49
avp:  0.5362746722894707
	Waste
acc:  0.285
avp:  0.35522489766840765
	Law
acc:  0.205
avp:  0.20375176203407225


In [6]:
## inter annotator agreement
a_nc = DIR1 + "annotatore NC.xlsx"
a_me = DIR1 + "annotatore ME.xlsx"
a_ag = DIR1 + "annotatore AG.xlsx"
annot = [a_ag, a_me, a_nc]
fleiss(annot, col="Dominio")

0.47583914451815496